## Assignment 3: $k$ Nearest Neighbor

**Do two questions.**

`! git clone https://github.com/DS3001/knn`

In [236]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt


**Q1.** This question is a case study for $k$ nearest neighbor The target variable `y` is `price` and the features are `year` and `mileage`.

1. Load the `./data/USA_cars_datasets.csv`. Keep the following variables and drop the rest: `price`, `year`, `mileage`. Are there any `NA`'s to handle? Look at the head and dimensions of the data.
2. Maxmin normalize `year` and `mileage`.
3. Split the sample into ~80% for training and ~20% for evaluation.
4. Use the $k$NN algorithm and the training data to predict `price` using `year` and `mileage` for the test set for $k=3,10,25,50,100,300$. For each value of $k$, compute the mean squared error and print a scatterplot showing the test value plotted against the predicted value. What patterns do you notice as you increase $k$?
5. Determine the optimal $k$ for these data.
6. Describe what happened in the plots of predicted versus actual prices as $k$ varied, taking your answer into part 6 into account. (Hint: Use the words "underfitting" and "overfitting".)

In [237]:
df = pd.read_csv("./knn/data/USA_cars_datasets.csv")

In [238]:
df = df.loc[:, ["price", "year", "mileage"]]

In [239]:
columns = ["price", "year", "mileage"]

print(df[columns].isna().sum())
print(df.shape)

price      0
year       0
mileage    0
dtype: int64
(2499, 3)


In [250]:
def maxmin(z):
    z = ( z-min(z) ) / ( max(z)-min(z) )
    return z

df['year'] = maxmin(df['year'])
df['mileage'] = maxmin(df['mileage'])

KeyError: 'year'

In [ ]:
y = df['price']
X = df.drop('price',axis=1 )

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=100)

In [ ]:
test_set = [3, 10, 23, 50, 100, 300]
SSE = np.zeros(6)
loc = 0

for num in test_set:
  print(num)
  model = KNeighborsRegressor(n_neighbors = num).fit(X_train, y_train)
  y_hat = model.predict(X_test)
  SSE[loc] = np.sum((y_test - y_hat )**2)
  print("The SSE this was", SSE[loc], "at", num, "neighbors")
  plt.scatter(y_test, y_hat)
  plt.show()

  loc += 1


As you increase K high enough the values on the extremes become impossible to achieve.

In [ ]:
k_bar = 200
SSE = np.zeros(k_bar)
k_grid = np.arange(1,k_bar)

for k in range(k_bar):
    model = KNeighborsRegressor(n_neighbors=k+1).fit(X_train, y_train)
    y_hat = model.predict(X_test)
    SSE[k] = np.sum( (y_test - y_hat)**2 )


lowest_SSE = np.min(SSE)
corresponding_K = np.where(SSE == lowest_SSE)
k_star = k_grid[corresponding_K]
print("The optimal amount of neighbors is k = ", k_star)

plt.plot(np.arange(0, k_bar), SSE)
plt.xlabel("K")
plt.ylabel("SSE")



Changing the number of neighbors greatly influences the accuracy of this model. At lower K values such as 3 and 10, there is underfitting. This is in contrast with high K values like 300 where overfitting takes place (We are looking at too many neighbors relative to the size of the dataset to make meaningful predictions). The best K value is a k of 77

**Q2.** This question is a case study for $k$ nearest neighbor. The data for the question include:

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)
- death event: if the patient deceased during the follow-up period (boolean)

1. Load the `./data/heart_failure_clinical_records_dataset.csv`. Are there any `NA`'s to handle? use `.drop()` to remove `time` from the dataframe.
2. Make a correlation matrix. What variables are strongly associated with a death event?
3. For the dummy variables `anaemia`, `diabetes`, `high_blood_pressure`, `sex`, and `smoking`, compute a summary table of `DEATH_EVENT` grouped by the variable. For which variables does a higher proportion of the population die when the variable takes the value 1 rather than 0?
4. On the basis of your answers from 2 and 3, build a matrix $X$ of the variables you think are most predictive of a death, and a variable $y$ equal to `DEATH_EVENT`.
5. Maxmin normalize all of the variables in `X`.
6. Split the sample into ~80% for training and ~20% for evaluation. (Try to use the same train/test split for the whole question, so that you're comparing apples to apples in the questions below.).
7. Determine the optimal number of neighbors for a $k$NN regression for the variables you selected.
8. OK, do steps 5 through 7 again, but use all of the variables (except `time`). Which model has a lower Sum of Squared Error? Which would you prefer to use in practice, if you had to predict `DEATH_EVENT`s? If you play with the selection of variables, how much does the SSE change for your fitted model on the test data? Are more variables always better? Explain your findings.

In [244]:
df = pd.read_csv("./knn/data/heart_failure_clinical_records_dataset.csv")

In [ ]:
df = df.drop("time", axis = 1)
df.isna().sum()

In [246]:
df.corr()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,DEATH_EVENT
age,1.000000,0.088006,-0.081584,-0.101012,0.060098,0.093289,-0.052354,0.159187,-0.045966,0.065430,0.018668,0.253729
anaemia,0.088006,1.000000,-0.190741,-0.012729,0.031557,0.038182,-0.043786,0.052174,0.041882,-0.094769,-0.107290,0.066270
creatinine_phosphokinase,-0.081584,-0.190741,1.000000,-0.009639,-0.044080,-0.070590,0.024463,-0.016408,0.059550,0.079791,0.002421,0.062728
diabetes,-0.101012,-0.012729,-0.009639,1.000000,-0.004850,-0.012732,0.092193,-0.046975,-0.089551,-0.157730,-0.147173,-0.001943
ejection_fraction,0.060098,0.031557,-0.044080,-0.004850,1.000000,0.024445,0.072177,-0.011302,0.175902,-0.148386,-0.067315,-0.268603
high_blood_pressure,0.093289,0.038182,-0.070590,-0.012732,0.024445,1.000000,0.049963,-0.004935,0.037109,-0.104615,-0.055711,0.079351
platelets,-0.052354,-0.043786,0.024463,0.092193,0.072177,0.049963,1.000000,-0.041198,0.062125,-0.125120,0.028234,-0.049139
serum_creatinine,0.159187,0.052174,-0.016408,-0.046975,-0.011302,-0.004935,-0.041198,1.000000,-0.189095,0.006970,-0.027414,0.294278
serum_sodium,-0.045966,0.041882,0.059550,-0.089551,0.175902,0.037109,0.062125,-0.189095,1.000000,-0.027566,0.004813,-0.195204
sex,0.065430,-0.094769,0.079791,-0.157730,-0.148386,-0.104615,-0.125120,0.006970,-0.027566,1.000000,0.445892,-0.004316


In [247]:
for val in ['anaemia','diabetes','high_blood_pressure','sex','smoking']:
  print(df.loc[ : , ['DEATH_EVENT', val]].groupby(val).describe())

        DEATH_EVENT                                             
              count      mean       std  min  25%  50%  75%  max
anaemia                                                         
0             170.0  0.294118  0.456991  0.0  0.0  0.0  1.0  1.0
1             129.0  0.356589  0.480859  0.0  0.0  0.0  1.0  1.0
         DEATH_EVENT                                             
               count      mean       std  min  25%  50%  75%  max
diabetes                                                         
0              174.0  0.321839  0.468530  0.0  0.0  0.0  1.0  1.0
1              125.0  0.320000  0.468353  0.0  0.0  0.0  1.0  1.0
                    DEATH_EVENT                                             
                          count      mean       std  min  25%  50%  75%  max
high_blood_pressure                                                         
0                         194.0  0.293814  0.456687  0.0  0.0  0.0  1.0  1.0
1                         105.0  0.37

High blood pressure appears to be the best predictor out of the variables included. Patients without it have a survival rate of 29% on average versus 37% with. The second best predictor is anaemia.

In [248]:
X = df.loc[: , ['age', 'ejection_fraction', 'high_blood_pressure', 'anaemia']]
y = df["DEATH_EVENT"]

In [ ]:
def maxmin(z):
    z = ( z-min(z) ) / ( max(z)-min(z) )
    return z

X = X.apply(maxmin)

In [ ]:
np.random.seed(100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

k_bar = 30
k_vals = np.arange(1,k_bar)
SSE = np.zeros(k_bar)

for k in range(k_bar):
    knn = KNeighborsRegressor(n_neighbors=k+1)
    predictor = knn.fit(X_train,y_train)
    y_hat = knn.predict(X_test)
    SSE[k] = np.sum( ( y_test-y_hat ) **2 )

SSE_min = np.min(SSE)
min = np.where(SSE==SSE_min)
k_star = k_vals[min]
print("Optimal K of :", k_star)

plt.plot(np.arange(0,k_bar),SSE)
plt.xlabel("K")
plt.ylabel('SSE')

In [ ]:
X = df.drop('DEATH_EVENT', axis = 1)

k_bar = 100
k_vals = np.arange(1,k_bar)
SSE = np.zeros(k_bar)

for k in range(k_bar):
    knn = KNeighborsRegressor(n_neighbors=k+1)
    predictor = knn.fit(X_train,y_train)
    y_hat = knn.predict(X_test)
    SSE[k] = np.sum( ( y_test-y_hat ) **2 )

SSE_min = np.min(SSE)
min = np.where(SSE==SSE_min)
k_star = k_vals[min]
print("Optimal K of :", k_star)

plt.plot(np.arange(0,k_bar),SSE)
plt.xlabel("K")
plt.ylabel('SSE')

**Q3.** Let's do some very basic computer vision. We're going to import the MNIST handwritten digits data and $k$NN to predict values (i.e. "see/read").

1. To load the data, run the following code in a chunk:
```
from keras.datasets import mnist
df = mnist.load_data('minst.db')
train,test = df
X_train, y_train = train
X_test, y_test = test
```
The `y_test` and `y_train` vectors, for each index `i`, tell you want number is written in the corresponding index in `X_train[i]` and `X_test[i]`. The value of `X_train[i]` and `X_test[i]`, however, is a 28$\times$28 array whose entries contain values between 0 and 256. Each element of the matrix is essentially a "pixel" and the matrix encodes a representation of a number. To visualize this, run the following code to see the first ten numbers:
```
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)
for i in range(5):
    print(y_test[i],'\n') # Print the label
    print(X_test[i],'\n') # Print the matrix of values
    plt.contourf(np.rot90(X_test[i].transpose())) # Make a contour plot of the matrix values
    plt.show()
```
OK, those are the data: Labels attached to handwritten digits encoded as a matrix.

2. What is the shape of `X_train` and `X_test`? What is the shape of `X_train[i]` and `X_test[i]` for each index `i`? What is the shape of `y_train` and `y_test`?
3. Use Numpy's `.reshape()` method to covert the training and testing data from a matrix into an vector of features. So, `X_test[index].reshape((1,784))` will convert the $index$-th element of `X_test` into a $28\times 28=784$-length row vector of values, rather than a matrix. Turn `X_train` into an $N \times 784$ matrix $X$ that is suitable for scikit-learn's kNN classifier where $N$ is the number of observations and $784=28*28$ (you could use, for example, a `for` loop).
4. Use the reshaped `X_test` and `y_test` data to create a $k$-nearest neighbor classifier of digit. What is the optimal number of neighbors $k$? If you can't determine this, play around with different values of $k$ for your classifier.
5. For the optimal number of neighbors, how well does your predictor perform on the test set?
6. So, this is how computers "see." They convert an image into a matrix of values, that matrix becomes a vector in a dataset, and then we deploy ML tools on it as if it was any other kind of tabular data. To make sure you follow this, invent a way to represent a color photo in matrix form, and then describe how you could convert it into tabular data. (Hint: RGB color codes provide a method of encoding a numeric value that represents a color.)